In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_predict
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from scipy.sparse import csr_matrix
from scipy.sparse import hstack

In [8]:
path = "/Users/aryagupta/Desktop/yt-sentiment-analysis/youtube-comments-sentiment-analysis/data/processed/preprocessed_v1.csv"

data = pd.read_csv(path)

In [9]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [10]:
mlflow.set_experiment("Experiment 2.1 BoW vs TF-IDF")

2025/05/08 10:15:39 INFO mlflow.tracking.fluent: Experiment with name 'Experiment 2.1 BoW vs TF-IDF' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/293310681361475325', creation_time=1746679539502, experiment_id='293310681361475325', last_update_time=1746679539502, lifecycle_stage='active', name='Experiment 2.1 BoW vs TF-IDF', tags={}>

In [11]:
data.shape

(36793, 2)

In [12]:
def run_experiment(vectorizer_type, ngram_range, vectorizer_max_feature, vectorizer_name):
    X_raw_train, X_raw_test, y_train, y_test = train_test_split(
        data['clean_comment'], data['category'], test_size=0.2, random_state=1, stratify=data['category']
    )

    if vectorizer_type == "BoW":
        vectorizer = CountVectorizer(ngram_range=ngram_range, max_features=vectorizer_max_feature)
    else:
        vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=vectorizer_max_feature)

    X_train = vectorizer.fit_transform(X_raw_train)
    X_test = vectorizer.transform(X_raw_test)

    with mlflow.start_run() as run:
        mlflow.set_tag("mlflow.runName", f"{vectorizer_name}_{ngram_range}_RandomForest")
        mlflow.set_tag("experiment_type", "feature_engineering")
        mlflow.set_tag("model_type", "RandomForestClassifier")

        mlflow.set_tag(
            "description",
            f"RandomForest with {vectorizer_name}, ngram_range={ngram_range}, max_features={vectorizer_max_feature}"
        )

        mlflow.log_param("vectorizer_type", vectorizer_type)
        mlflow.log_param("ngram_range", ngram_range)
        mlflow.log_param("vectorizer_max_feature", vectorizer_max_feature)

        n_estimators = 200
        max_depth = 15

        mlflow.log_param("n_estimators", n_estimators)
        mlflow.log_param("max_depth", max_depth)

        model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=1)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_test)

        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy_score", accuracy)

        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        conf_matrix = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(8, 6))
        sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title(f"Confusion Matrix: {vectorizer_name}, {ngram_range}")
        plot_path = "/Users/aryagupta/Desktop/yt-sentiment-analysis/youtube-comments-sentiment-analysis/plots/confusion_matrix.png"
        plt.savefig(plot_path)
        mlflow.log_artifact(plot_path)
        plt.close()

        mlflow.sklearn.log_model(model, f"random_forest_model_{vectorizer_name}_{ngram_range}")

In [13]:
ngram_ranges = [(1, 1), (1, 2), (1, 3)]
max_features = 5000

for ngram_range in ngram_ranges:
    run_experiment("BoW", ngram_range, max_features, vectorizer_name="BoW")
    run_experiment("TF-IDF", ngram_range, max_features, vectorizer_name="TF-IDF")

2025/05/08 10:16:33 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run BoW_(1, 1)_RandomForest at: http://127.0.0.1:5000/#/experiments/293310681361475325/runs/fd176dc6af2342158fafb9aeda5e23d4
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/293310681361475325


2025/05/08 10:16:39 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run TF-IDF_(1, 1)_RandomForest at: http://127.0.0.1:5000/#/experiments/293310681361475325/runs/3f4efdea01e742b8ac387c8e38c311b2
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/293310681361475325


2025/05/08 10:16:45 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


...

🏃 View run TF-IDF_(1, 2)_RandomForest at: http://127.0.0.1:5000/#/experiments/293310681361475325/runs/bc077db2efe34d859c26fbdb10331513
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/293310681361475325


2025/05/08 10:17:00 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run BoW_(1, 3)_RandomForest at: http://127.0.0.1:5000/#/experiments/293310681361475325/runs/fa910211fec84789b7b4a8ec6ecac6c3
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/293310681361475325


2025/05/08 10:17:08 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run TF-IDF_(1, 3)_RandomForest at: http://127.0.0.1:5000/#/experiments/293310681361475325/runs/5cd356408723413eb13737e722f34583
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/293310681361475325


![MLflow Experiment Screenshot](https://raw.githubusercontent.com/realaryagupta/youtube-comment-sentiment-analysis/master/plots/screenshots/Exp-2-mlflow.png)

On careful observing we select TF-IDF with Trigram setting as this is the best we can get